<a href="https://colab.research.google.com/github/KLeeDE/ResearchProj_UrbanHeat/blob/master/SaveIMGtoPIX/20210216_2_making_feather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# -- get the file list
fname = "file_paths_times.feather"
if os.path.isfile(fname):
  print("reading {0}".format(fname))
  fpaths = pd.read_feather(fname)
else:
  print("{0} not found... creating".format(fname))
  fldrs = sorted([i for i in glob.glob("*-*") if len(i) == 5])
  # -- extract the time and convert to datetime
  fmt = "%Y-%m-%d %H-%M-%S.%f"
  flist = []
  ftime = []
  for fldr in fldrs:
    print("\rworking on {0}".format(fldr), end="")
    tflist = sorted(glob.glob(os.path.join(fldr, "*")))
    flist += tflist
    ftime += [pd.to_datetime(i[11:-5].replace("T", " "), format=fmt) for i in tflist]
  # -- create pandas data frame
  fpaths = pd.DataFrame()
  fpaths["paths"] = flist
  fpaths["times"] = ftime
  # -- add numpy file names
  nfile = int(np.ceil(len(fpaths) / 500))
  fnames_npy = [jj for ii in range(nfile) for jj in 
                ["singapore_ir_npy/singapore_ir_{0:06}_{1:06}.npy" \
                 .format(ii * 500, (ii + 1) * 500)] * 500]
  fpaths["fnames_npy"] = fnames_npy[:len(fpaths)]
  # -- write all paths to feather
  fpaths.to_feather("file_paths_times.feather")